In [1]:
import pandas as pd 
import pickle
import matplotlib.pyplot as plt 
import seaborn as sns
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression, Lasso, Ridge 
from sklearn.metrics import mean_squared_error
import mlflow
from pathlib import Path

import xgboost as xgb 
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from  hyperopt.pyll import scope

In [2]:
# specify mlflow -- use local stores for now
mlflow.set_tracking_uri("sqlite:///mlflow.db")
mlflow.set_experiment("nyc-taxi-experiment")

<Experiment: artifact_location='/Users/addingtongraham/github-repos/mlops/nyc-taxi/notebooks/mlruns/1', creation_time=1685637916060, experiment_id='1', last_update_time=1685637916060, lifecycle_stage='active', name='nyc-taxi-experiment', tags={}>

In [3]:
# function to read parquet datasets
def read_dataframe(filename):
    df = pd.read_parquet(filename)
    
    # preprocess 
    df.lpep_dropoff_datetime = pd.to_datetime(df.lpep_dropoff_datetime)
    df.lpep_pickup_datetime = pd.to_datetime(df.lpep_pickup_datetime)
    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime 
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)
    df = df[(df.duration >= 1) & (df.duration <= 60)]
    
    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    return df

In [4]:
# pull training and validation sets
data_path = Path('../data/')
df_train = read_dataframe(filename = data_path/'green_tripdata_2022-01.parquet')
df_val = read_dataframe(filename = data_path/'green_tripdata_2022-02.parquet')
len(df_train), len(df_val)

(59603, 66097)

In [5]:
# combine pick up and drop off into one feature
df_train['PU_DO'] = df_train['PULocationID'] + '-' + df_train['DOLocationID']
df_val['PU_DO'] = df_val['PULocationID'] + '-' + df_val['DOLocationID']

In [6]:
categorical = ['PU_DO']
numerical = ['trip_distance']

dv = DictVectorizer()

train_dicts = df_train[categorical + numerical].to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)

val_dicts = df_val[categorical + numerical].to_dict(orient='records')
X_val = dv.transform(val_dicts)

In [7]:
type(X_train) # sparse matrix

scipy.sparse._csr.csr_matrix

In [8]:
# if models can handle sparse arrays, we can skip this
target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values

In [9]:
type(y_train)

numpy.ndarray

In [10]:
# set up linear regression
lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_val)

mean_squared_error(y_val, y_pred, squared=False)

6.928931234319288

In [11]:
# ensure models folder exists 
Path('../models').mkdir(exist_ok=True)

# save model
with open('../models/lin_reg.bin', 'wb') as f_out:
    pickle.dump((dv, lr), f_out)

In [12]:
# mlflow run
with mlflow.start_run():
    mlflow.set_tag('developer', 'akg')
    mlflow.log_param('train-data-path', '../data/green_tripdata_2022-01.parquet')
    mlflow.log_param('valid-data-path', '../data/green_tripdata_2022-02.parquet')
    
    alpha = 0.1
    mlflow.log_param('alpha', alpha)
    lr = Lasso(alpha)
    lr.fit(X_train, y_train)
    
    y_pred = lr.predict(X_val)
    rmse = mean_squared_error(y_val, y_pred, squared=False)
    mlflow.log_metric('rmse', rmse)
    
    mlflow.log_artifact(local_path='../models/lin_reg.bin', artifact_path="models_pickle")

In [14]:
# train model with xgboost and hyperparam tuning
train = xgb.DMatrix(X_train, label=y_train)
valid = xgb.DMatrix(X_val, label=y_val)

In [15]:
# specify objective to optimize 
def objective(params):
    with mlflow.start_run():
        mlflow.set_tag('model', 'xgboost')
        mlflow.log_params(params)
        booster = xgb.train(
            params=params,
            dtrain=train,
            num_boost_rounds=200,
            evals=[(valid, 'validation')],
            early_stopping_rounds=50
        )
        y_pred = booster.predict(valid)
        rmse = mean_squared_error(y_val, y_pred, squared=False)
        mlflow.log_metric('rmse', rmse)
    return {'loss': rmse, 'status': STATUS_OK}

In [ ]:
# specify search space
search_space = {
    'max_depth': scope.int(hp.quniform('max_depth', 4, 100, 1)),
    'learning_rate': hp.loguniform('learning_rate', -3, 0),
    'reg_alpha': hp.loguniform('reg_alpha', -5, -1),
    'reg_lambda': hp.loguniform('reg_lambda', -6, -1),
    'min_child_weight': hp.loguniform('min_child_weight', -1, 3),
    'objective': 'reg:linear',
    'seed': 23
}

# best results
best_result = fmin(
    fn=objective,
    space=search_space,
    algo=tpe.suggest,
    max_evals=50,
    trials=Trials()
)